# Hold Predictions

In [12]:
import pandas as pd
import numpy as np
import joblib

## Read dataset and complete same transitions as training and test set

In [2]:
df = pd.read_csv('holdout_data.csv' , index_col=0)

In [3]:
df = df.rename(columns=
           {'X1': 'given_credit',
            'X2': 'gender',
            'X3': 'education',
            'X4': 'martial_status',
            'X5': 'age',
            'X6': 'past_status_sept',
            'X7': 'past_status_aug',
            'X8': 'past_status_jul',
            'X9': 'past_status_jun',
           'X10': 'past_status_may',
           'X11': 'past_status_apr',
           'X12': 'bill_sept',
           'X13': 'bill_aug',
           'X14': 'bill_jul',
           'X15': 'bill_jun',
           'X16': 'bill_may',
           'X17': 'bill_apr',
           'X18': 'paid_sept',
           'X19': 'paid_aug',
           'X20': 'paid_jul',
           'X21': 'paid_jun',
           'X22': 'paid_may',
           'X23': 'paid_apr',
             'Y': 'target'},
              inplace=False)

In [4]:
df = df.apply(pd.to_numeric, errors='coerce')

In [5]:
df.dropna(axis=0, inplace=True)

In [6]:
df['education'] = df['education'].replace(0, 4)
df['education'] = df['education'].replace(5, 4)
df['education'] = df['education'].replace(6, 4)

In [7]:
df['martial_status'] = df['martial_status'].replace(0, 3)

In [8]:
df.loc[df['given_credit'] >= 800000, 'given_credit'] = 800000

In [9]:
# ADJUSTING BILL OUTLIERS
df.loc[df['bill_sept'] >= 450000, 'bill_sept'] = 450000
df.loc[df['bill_sept'] <= -9000, 'bill_sept'] = -8000
df.loc[df['bill_aug'] >= 450000, 'bill_aug'] = 450000
df.loc[df['bill_jul'] >= 450000, 'bill_jul'] = 450000
df.loc[df['bill_jul'] <= -100000, 'bill_jul'] = -80000
df.loc[df['bill_jun'] >= 450000, 'bill_jun'] = 450000
df.loc[df['bill_may'] >= 450000, 'bill_may'] = 450000
df.loc[df['bill_apr'] >= 450000, 'bill_apr'] = 450000
df.loc[df['bill_apr'] <= -100000, 'bill_apr'] = -150000

In [10]:
df.loc[df['paid_sept'] >= 100000, 'paid_sept'] = 100000
df.loc[df['paid_aug'] >= 250000, 'paid_aug'] = 250000
df.loc[df['paid_jul'] >= 210000, 'paid_jul'] = 210000
df.loc[df['paid_jun'] >= 210000, 'paid_jun'] = 210000
df.loc[df['paid_may'] >= 210000, 'paid_may'] = 210000
df.loc[df['paid_apr'] >= 210000, 'paid_apr'] = 210000

In [13]:
df['martial_status_married'] = np.where((df['martial_status']==1), 1, 0)
df['martial_status_single'] = np.where((df['martial_status']==2), 1, 0)

In [14]:
df['edu_gradschool'] = np.where((df['education']==1), 1, 0)
df['edu_uni'] = np.where((df['education']==2), 1, 0)
df['edu_highschool'] = np.where((df['education']==3), 1, 0)

In [15]:
df['past_status_sum'] = (df.past_status_sept + df.past_status_aug + df.past_status_jul + df.past_status_jun + df.past_status_may + df.past_status_apr) 
df['past_status_mean'] = ((df.past_status_sept + df.past_status_aug + df.past_status_jul + df.past_status_jun + df.past_status_may + df.past_status_apr) / 6)

In [16]:
df['bill_sum'] = (df.bill_sept + df.bill_aug + df.bill_jul + df.bill_jun + df.bill_may + df.bill_apr) 
df['bill_mean'] = ((df.bill_sept + df.bill_aug + df.bill_jul + df.bill_jun + df.bill_may + df.bill_apr) / 6)

In [17]:
df['paid_sum'] = (df.paid_sept + df.paid_aug + df.paid_jul + df.paid_jun + df.paid_may + df.paid_apr) 
df['paid_mean'] = ((df.paid_sept + df.paid_aug + df.paid_jul + df.paid_jun + df.paid_may + df.paid_apr) / 6)

In [18]:
df['bill_paid_sum_diff'] = df.bill_sum - df.paid_sum
df['bill_paid_mean_diff'] = df.bill_mean - df.paid_mean

In [19]:
df['never_late'] = np.where((df['past_status_sept']<=0) & 
                    (df['past_status_aug']<=0) & 
                    (df['past_status_jul']<=0) &
                    (df['past_status_jun']<=0) & 
                    (df['past_status_may']<=0) & 
                    (df['past_status_apr']<=0), 1, 0)

## Predict on holdout set

In [20]:
joblib_file = "bagged_dtc.pkl"  
final_model = joblib.load(joblib_file)
output = final_model.predict(df)

## Export

In [22]:
final_answer = pd.DataFrame(output, columns=['predictions']).to_csv('credit_preds_amelia_dahm.csv')
